<a href="https://colab.research.google.com/github/INFINITY-RUBER/Curso_Deep_Learning_Con_TernsorFlow_Machine-Learning_e_IA-/blob/master/scripts/tema06/05-address-matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tensorflow==1.14

     |████████████████████████████████| 109.2MB 74kB/s 
     |████████████████████████████████| 491kB 38.0MB/s 
     |████████████████████████████████| 3.2MB 41.4MB/s 
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0


# Correspondencia en direcciones postales

In [0]:
import random
import string
import numpy as np
import tensorflow as tf

In [0]:
n = 10
street_names = ["diagon", "elm", "abbey", "gran", "python"]
street_type = ["callejon", "calle", "carrera", "via", "avenida"]
street_zips = [random.randint(20000, 29999) for i in range(5)]
numbers = [random.randint(1,999) for i in range(n)] # numeros aleatorios enteros

In [0]:
streets = [random.choice(street_names) for i in range(n)]
street_prefs = [random.choice(street_type) for i in range(n)]
zips = [random.choice(street_zips) for i in range(n)]
full_streets = [x + " " + y + " " + str(z) for x,y,z in zip(street_prefs, streets, numbers)]# Junta y recorre tres Array
reference_data = [list(x) for x in zip(full_streets, zips)]

In [9]:
reference_data

[['calle abbey 556', 21023],
 ['avenida abbey 853', 21023],
 ['calle abbey 824', 23032],
 ['callejon abbey 388', 24682],
 ['avenida elm 803', 21023],
 ['avenida elm 258', 24682],
 ['carrera abbey 750', 21023],
 ['calle elm 202', 24682],
 ['carrera abbey 575', 25808],
 ['callejon abbey 825', 23032]]

In [0]:
def create_typo(s, prob=0.75):
    if random.uniform(0,1)<0.75:
        rand_idx = random.choice(range(len(s)))
        s_list = list(s)
        s_list[rand_idx] = random.choice(string.ascii_lowercase)# string.ascii_lowercase (cualquier letra en minuscuala)
        s = ''.join(s_list)
    return s

In [0]:
typo_streets = [create_typo(x) for x in streets]

In [39]:
typo_streets

['abbey',
 'apbey',
 'abbxy',
 'abbey',
 'evm',
 'plm',
 'abbgy',
 'klm',
 'abbeh',
 'abbey']

In [0]:
typo_full_streets = [x+" "+y+" "+str(z) for x,y,z in zip(street_prefs, typo_streets, numbers)]
test_data = [list(x) for x in zip(typo_full_streets, zips)]

In [19]:
test_data

[['calle abbey 556', 21023],
 ['avenida apbey 853', 21023],
 ['calle abbxy 824', 23032],
 ['callejon abbey 388', 24682],
 ['avenida evm 803', 21023],
 ['avenida plm 258', 24682],
 ['carrera abbgy 750', 21023],
 ['calle klm 202', 24682],
 ['carrera abbeh 575', 25808],
 ['callejon abbey 825', 23032]]

In [0]:
session = tf.Session()

In [0]:
test_address = tf.sparse_placeholder(dtype = tf.string)# contendra las palabras de la direccion
test_zip = tf.placeholder(shape = [None, 1], dtype=tf.float32)# codigo postal es un numero

ref_address = tf.sparse_placeholder(dtype = tf.string)  # contendra las palabras de la direccion de referencia
ref_zip = tf.placeholder(shape=[None, n], dtype=tf.float32 )

In [0]:
zip_dist = tf.square(tf.subtract(ref_zip, test_zip))
address_dist = tf.edit_distance(test_address, ref_address, normalize=True)

- $S(x,y) = 0$ si $x$ e $y$ son totalmente diferentes (no se parecen en nada)
- $S(x,x) = 1$, ya que todo objeto es similar (si no igual) a si mismo.
- $S(x,y) = \frac{D - d(x,y)}{D-d}$ donde $D$ es la mayor distancia entre dos objetos posibles, y $d$ es la menor.

In [0]:
zip_max = tf.gather(tf.squeeze(zip_dist), tf.argmax(zip_dist, 1))
zip_min = tf.gather(tf.squeeze(zip_dist), tf.argmin(zip_dist, 1))


- $S(x,y) = \frac{D - d(x,y)}{D-d}$ donde $D$ es la mayor distancia entre dos objetos posibles, y $d$ es la menor.

In [0]:
zip_sim = tf.divide(tf.subtract(zip_max, zip_dist), tf.subtract(zip_max, zip_min))# similaridad

In [0]:
address_sim = tf.subtract(1.0, address_dist)

### SIMILARIDAD

$$S(x,y) = \sum_{i=1}^k w_iS_k(x,y):\quad \sum_{i=1}^kw_i = 1$$

In [0]:
address_wi = 0.5# PESO
zip_wi = 1.0 -address_wi

In [0]:
# Similarada ponderada
weighted_sim = tf.add(tf.transpose(tf.multiply(address_wi, address_sim)),tf.multiply(zip_wi, zip_sim))# se traspone por que es un vector

In [0]:
# EL VECIONO MAS CERCANO
top_match_idx = tf.argmax(weighted_sim, 1)

In [0]:
# OBJETOS NECESARIOS PARA EL ALGORIMOS
def sparse_from_word_vector(word_vector):
    num_words = len(word_vector)
    idx = [[xi, 0, yi] for xi, x in enumerate(word_vector) for yi, y in enumerate(x)]
    chars = list(''.join(word_vector))# genera una lista con las letras del elemento
    return tf.SparseTensorValue(idx, chars, [num_words,1,1])

In [0]:
reference_address = [x[0] for x in reference_data]
reference_zips = np.array([[x[1] for x in reference_data]])

In [0]:
sparse_ref_set = sparse_from_word_vector(reference_address)

In [38]:
for i in range(n):
    test_address_entry = test_data[i][0]
    test_zip_entry = [[test_data[i][1]]]
    
    test_address_rep = [test_address_entry]*n
    sparse_test_set = sparse_from_word_vector(test_address_rep)
    
    feed_dict = {test_address: sparse_test_set,
                test_zip: test_zip_entry,
                ref_address: sparse_ref_set,
                ref_zip: reference_zips}
    
    best_match = session.run(top_match_idx, feed_dict=feed_dict)# GENERA EL VECINO UNICO CERCANO
    best_address = reference_address[best_match[0]]
    [best_zip] = reference_zips[0][best_match]
    [[test_zip_aux]] = test_zip_entry
    
    print("Dirección original: "+str(test_address_entry)+ ", "+str(test_zip_aux))
    print("Dirección corregida: "+str(best_address)+", "+str(best_zip)+"\n")

Dirección original: calle abbey 556, 21023
Dirección corregida: calle abbey 556, 21023

Dirección original: avenida apbey 853, 21023
Dirección corregida: avenida abbey 853, 21023

Dirección original: calle abbxy 824, 23032
Dirección corregida: calle abbey 824, 23032

Dirección original: callejon abbey 388, 24682
Dirección corregida: callejon abbey 388, 24682

Dirección original: avenida evm 803, 21023
Dirección corregida: avenida elm 803, 21023

Dirección original: avenida plm 258, 24682
Dirección corregida: avenida elm 258, 24682

Dirección original: carrera abbgy 750, 21023
Dirección corregida: carrera abbey 750, 21023

Dirección original: calle klm 202, 24682
Dirección corregida: calle elm 202, 24682

Dirección original: carrera abbeh 575, 25808
Dirección corregida: carrera abbey 575, 25808

Dirección original: callejon abbey 825, 23032
Dirección corregida: callejon abbey 825, 23032

